In [1]:
#import libraries

import numpy as np
from numpy import nan
from xgboost import XGBClassifier
import sklearn

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_roc_curve

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#train data
train = pd.read_csv("https://raw.githubusercontent.com/SupreethBaliga/SOC471_Proj/main/census-income.data?token=AMEP55QZFUDGA35WYKX47G3BRDCTK",header=None)
cols = ['age', 'class of worker', 'detailed industry recode',
       'detailed occupation recode', 'education', 'wage per hour',
       'enroll in edu inst last wk', 'marital stat', 'major industry code',
        'major occupation code', 'race', 'hispanic origin', 'sex',
       'member of a labor union', 'reason for unemployment', 'full or part time employment stat',
       'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'region of previous residence',
       'state of previous residence', 'detailed household and family stat', 'detailed household summary in household',
       'instance weight', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg',
       'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer',
       'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self',
       'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits',
       'weeks worked in year', 'survey year', 'Income Bracket']
train.columns = cols

train["Income Bracket"] = train["Income Bracket"].map({" - 50000.": 0, ' 50000+.':1})
train["Income Bracket"] = train["Income Bracket"].astype(int)

train.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,instance weight,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,survey year,Income Bracket
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,0
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,0
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0


In [3]:
#test data
test = pd.read_csv("https://raw.githubusercontent.com/SupreethBaliga/SOC471_Proj/main/census-income.test?token=AMEP55WNLHPHOVQ24FNFYALBRDCT6",header=None)
cols = ['age', 'class of worker', 'detailed industry recode',
       'detailed occupation recode', 'education', 'wage per hour',
       'enroll in edu inst last wk', 'marital stat', 'major industry code',
        'major occupation code', 'race', 'hispanic origin', 'sex',
       'member of a labor union', 'reason for unemployment', 'full or part time employment stat',
       'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'region of previous residence',
       'state of previous residence', 'detailed household and family stat', 'detailed household summary in household',
       'instance weight', 'migration code-change in msa', 'migration code-change in reg', 'migration code-move within reg',
       'live in this house 1 year ago', 'migration prev res in sunbelt', 'num persons worked for employer',
       'family members under 18', 'country of birth father', 'country of birth mother', 'country of birth self',
       'citizenship', 'own business or self employed', "fill inc questionnaire for veteran's admin", 'veterans benefits',
       'weeks worked in year', 'survey year', 'Income Bracket']
test.columns = cols

test["Income Bracket"] = test["Income Bracket"].map({" - 50000.": 0, ' 50000+.':1})
test["Income Bracket"] = test["Income Bracket"].astype(int)

test.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,instance weight,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,survey year,Income Bracket
0,38,Private,6,36,1st 2nd 3rd or 4th grade,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Machine operators assmblrs & inspctrs,White,Mexican (Mexicano),Female,Not in universe,Not in universe,Full-time schedules,0,0,0,Joint one under 65 & one 65+,Not in universe,Not in universe,Spouse of householder,Spouse of householder,1032.38,?,?,?,Not in universe under 1 year old,?,4,Not in universe,Mexico,Mexico,Mexico,Foreign born- Not a citizen of U S,0,Not in universe,2,12,95,0
1,44,Self-employed-not incorporated,37,12,Associates degree-occup /vocational,0,Not in universe,Married-civilian spouse present,Business and repair services,Professional specialty,White,All other,Female,Not in universe,Not in universe,PT for econ reasons usually PT,0,0,2500,Joint both under 65,Not in universe,Not in universe,Spouse of householder,Spouse of householder,1462.33,?,?,?,Not in universe under 1 year old,?,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,26,95,0
2,2,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,Mexican-American,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1601.75,?,?,?,Not in universe under 1 year old,?,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,95,0
3,35,Private,29,3,High school graduate,0,Not in universe,Divorced,Transportation,Executive admin and managerial,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,Not in universe,Not in universe,Householder,Householder,1866.88,Nonmover,Nonmover,Nonmover,Yes,Not in universe,5,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,0
4,49,Private,4,34,High school graduate,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,0,Single,Not in universe,Not in universe,Secondary individual,Nonrelative of householder,1394.54,?,?,?,Not in universe under 1 year old,?,4,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,50,95,0


In [4]:
print(train.shape,test.shape)
comb = pd.concat([train,test])
print(comb.shape)

train_num = train.shape[0]
test_num = test.shape[0]
print(train_num, test_num)

(199523, 42) (99762, 42)
(299285, 42)
199523 99762


In [5]:
# Now our main task is to compute the frequencies of each value appearing in a column. 
cols = list(train)
colDict=dict()
for col in cols:
    colDict[col]=train[col].value_counts(sort=True).to_dict()
    print(col," : ",len(colDict[col])) #printing the number of unique values in each row

age  :  91
class of worker  :  9
detailed industry recode  :  52
detailed occupation recode  :  47
education  :  17
wage per hour  :  1240
enroll in edu inst last wk  :  3
marital stat  :  7
major industry code  :  24
major occupation code  :  15
race  :  5
hispanic origin  :  10
sex  :  2
member of a labor union  :  3
reason for unemployment  :  6
full or part time employment stat  :  8
capital gains  :  132
capital losses  :  113
dividends from stocks  :  1478
tax filer stat  :  6
region of previous residence  :  6
state of previous residence  :  51
detailed household and family stat  :  38
detailed household summary in household  :  8
instance weight  :  99800
migration code-change in msa  :  10
migration code-change in reg  :  9
migration code-move within reg  :  10
live in this house 1 year ago  :  3
migration prev res in sunbelt  :  4
num persons worked for employer  :  7
family members under 18  :  5
country of birth father  :  43
country of birth mother  :  43
country of birth 

In [6]:
#Selecting relevant columns: Tweak it later to choose fewer columns
#First row is for non-categorical variables ----> do not need one hot encoding

comb = comb[['age', 'wage per hour', 'instance weight', 'capital gains', 'capital losses', 'dividends from stocks', 'weeks worked in year', 
             'class of worker', 'detailed industry recode', 'detailed occupation recode', 'education', 'enroll in edu inst last wk',
             'marital stat', 'major industry code', 'major occupation code', 'race', 'hispanic origin', 'sex', 'member of a labor union', 'reason for unemployment', 
             'full or part time employment stat', "fill inc questionnaire for veteran's admin", 'own business or self employed', 'citizenship', 'country of birth self',
             'country of birth mother', 'country of birth father', 'family members under 18', 'num persons worked for employer', 'migration prev res in sunbelt','live in this house 1 year ago',
             'migration code-move within reg','migration code-change in reg','migration code-change in msa', 'detailed household summary in household', 'detailed household and family stat',
             'state of previous residence', 'region of previous residence','tax filer stat',
            'Income Bracket']]

print(comb.shape)

cols = list(comb)

(299285, 40)


In [7]:
# creating a mapping between column numbers and column names

num = dict()
j=0
for col in cols:
    num[j]=col
    j=j+1
print(num)    

{0: 'age', 1: 'wage per hour', 2: 'instance weight', 3: 'capital gains', 4: 'capital losses', 5: 'dividends from stocks', 6: 'weeks worked in year', 7: 'class of worker', 8: 'detailed industry recode', 9: 'detailed occupation recode', 10: 'education', 11: 'enroll in edu inst last wk', 12: 'marital stat', 13: 'major industry code', 14: 'major occupation code', 15: 'race', 16: 'hispanic origin', 17: 'sex', 18: 'member of a labor union', 19: 'reason for unemployment', 20: 'full or part time employment stat', 21: "fill inc questionnaire for veteran's admin", 22: 'own business or self employed', 23: 'citizenship', 24: 'country of birth self', 25: 'country of birth mother', 26: 'country of birth father', 27: 'family members under 18', 28: 'num persons worked for employer', 29: 'migration prev res in sunbelt', 30: 'live in this house 1 year ago', 31: 'migration code-move within reg', 32: 'migration code-change in reg', 33: 'migration code-change in msa', 34: 'detailed household summary in hou

In [9]:
comb.replace(np.nan, "No value",inplace=True)
comb.shape

(299285, 40)

In [11]:
# Compute the fraction of 'No value' values and ' ?' in our dataset

cols = list(comb)
total=0
nil1 =0
nil2 =0
for col in cols:
    total += comb[col].count() 
    nil1 += comb[(comb[col]=="No value")][col].count()
    nil2 += comb[(comb[col]==" ?")][col].count()

print('Fraction of No value values',nil1/total)
print('Fraction of ?',nil2/total)

Fraction of No value values 0.0
Fraction of ? 0.052132248525652805


In [12]:
# Converting features to one hot encodings

from pandas import Categorical
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

def sparse_dummies(df, column):
    """Returns sparse OHE matrix for the column of the dataframe"""
    categories = Categorical(df[column])
    column_names = np.array([f"{column}_{str(i)}" for i in range(len(categories.categories))])
    N = len(categories)
    row_numbers = np.arange(N, dtype=np.int)
    ones = np.ones((N,))
    return csr_matrix((ones, (row_numbers, categories.codes))), column_names

In [16]:
matrices = []
all_column_names = []
# creates a matrix per categorical feature

temp = list(comb)
ignore = ['age', 'wage per hour', 'instance weight', 'capital gains', 
  'capital losses', 'dividends from stocks', 'weeks worked in year','Income Bracket']
for c in temp:
  
    if c in ignore:
        continue  
    matrix, column_names = sparse_dummies(comb, c)
    matrices.append(matrix)
    all_column_names.append(column_names)

In [17]:
train_sparse = hstack(matrices, format="csr")
feature_names = np.concatenate(all_column_names)

In [18]:
# train_sparse stores the one hot encodings

print(train_sparse.shape, feature_names.shape)

(299285, 505) (505,)


In [19]:
import gc

X = train_sparse
y = comb['Income Bracket']

In [23]:
# Creating train and test data

import xgboost as xgb

x1, y1 = X[:train_num], y.iloc[:train_num]
dm1 = xgb.DMatrix(x1, y1, feature_names=feature_names)
dm1.save_binary('train_sparse1.bin')

x2, y2 = X[train_num:], y.iloc[train_num:]
dm2 = xgb.DMatrix(x2, y2, feature_names=feature_names)
dm2.save_binary('test_sparse1.bin')


In [24]:
from sklearn.linear_model import LogisticRegression

train_x, valid_x = csr_matrix(x1, dtype='float32'), csr_matrix(x2, dtype='float32')
train_y, valid_y = csr_matrix(y1, dtype='int16'), csr_matrix(y2, dtype='int16')

train_y = train_y.toarray()
train_y = train_y[0]
print(train_y.shape, train_x.shape)

valid_y = valid_y.toarray()
valid_y = valid_y[0]
print(valid_y.shape, valid_x.shape)

(199523,) (199523, 505)
(99762,) (99762, 505)


In [25]:
print(x1.shape,y1.shape,x2.shape,y2.shape)

(199523, 505) (199523,) (99762, 505) (99762,)


In [30]:
# Hyperparameter tuning 

from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_curve, auc, classification_report, accuracy_score, roc_auc_score

lgbm = LGBMClassifier(objective='binary', scale_pos_weight=12, max_depth=10)

params = {
        'n_estimators': range(200, 600, 50),
        #'gamma': list(np.linspace(0,2,50)),
        # 'gamma': [0, 0.5, 1, 1.5, 2, 4],
        #'subsample': [0.6, 0.8, 1.0],
        #'colsample_bytree': [0.8, 1.0],
        #'max_depth': range(7,15,1),
        #'lambda': list(np.linspace(0,2,50)),
        'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.3), base = 10, num = 1000)),
        'reg_alpha' : range(10,80,10),
        #'scale_pos_weight' : [7,8,9,10]
}

folds = 3
param_comb = 10
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state = 42)


print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, type(X_train))

random_search = RandomizedSearchCV(lgbm, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train, y_train), verbose = 3, random_state=42, refit='aucpr')
random_search.fit(X_train, y_train, eval_metric="aucpr", eval_set = [[X_val, y_val]])

print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

xgb_model = random_search

y_pred = xgb_model.predict(x2)
y_pred_proba = xgb_model.predict_proba(x2)[:, 1]

xgb_precision, xgb_recall, _ = precision_recall_curve(y2, y_pred_proba)
prscore = auc(xgb_recall, xgb_precision)

print(classification_report(y2, y_pred, digits = 5))
print('Accuracy: ', accuracy_score(y2, y_pred))
print('Roc-Auc score: ', roc_auc_score(y2, y_pred_proba))
print('Pr auc score: ', prscore)

(159618, 505) (159618,) (39905, 505) (39905,) <class 'scipy.sparse.csr.csr_matrix'>
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.8min finished


[1]	valid_0's binary_logloss: 0.195637
[2]	valid_0's binary_logloss: 0.191444
[3]	valid_0's binary_logloss: 0.191644
[4]	valid_0's binary_logloss: 0.19412
[5]	valid_0's binary_logloss: 0.197812
[6]	valid_0's binary_logloss: 0.202422
[7]	valid_0's binary_logloss: 0.206778
[8]	valid_0's binary_logloss: 0.211506
[9]	valid_0's binary_logloss: 0.215983
[10]	valid_0's binary_logloss: 0.220845
[11]	valid_0's binary_logloss: 0.225521
[12]	valid_0's binary_logloss: 0.230078
[13]	valid_0's binary_logloss: 0.234659
[14]	valid_0's binary_logloss: 0.239451
[15]	valid_0's binary_logloss: 0.243503
[16]	valid_0's binary_logloss: 0.247307
[17]	valid_0's binary_logloss: 0.251186
[18]	valid_0's binary_logloss: 0.254675
[19]	valid_0's binary_logloss: 0.258104
[20]	valid_0's binary_logloss: 0.261551
[21]	valid_0's binary_logloss: 0.264572
[22]	valid_0's binary_logloss: 0.267617
[23]	valid_0's binary_logloss: 0.270331
[24]	valid_0's binary_logloss: 0.272777
[25]	valid_0's binary_logloss: 0.275182
[26]	valid

In [28]:
print(colDict['Income Bracket'])

{0: 187141, 1: 12382}
